<a href="https://colab.research.google.com/github/alexmjn/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [5]:
df.head().T

,0,1,2,3,4
BOROUGH,1,1,1,1,1
NEIGHBORHOOD,OTHER,OTHER,OTHER,OTHER,UPPER EAST SIDE (59-79)
BUILDING_CLASS_CATEGORY,13 CONDOS - ELEVATOR APARTMENTS,21 OFFICE BUILDINGS,21 OFFICE BUILDINGS,13 CONDOS - ELEVATOR APARTMENTS,15 CONDOS - 2-10 UNIT RESIDENTIAL
TAX_CLASS_AT_PRESENT,2,4,4,2,2C
BLOCK,716,812,839,592,1379
LOT,1246,68,69,1041,1402
EASE-MENT,NaN,NaN,NaN,NaN,NaN
BUILDING_CLASS_AT_PRESENT,R4,O5,O5,R4,R1
ADDRESS,"447 WEST 18TH STREET, PH12A",144 WEST 37TH STREET,40 WEST 38TH STREET,"1 SHERIDAN SQUARE, 8C","20 EAST 65TH STREET, B"
APARTMENT_NUMBER,PH12A,NaN,NaN,8C,B


In [0]:
df = df[df["BUILDING_CLASS_CATEGORY"] == '01 ONE FAMILY DWELLINGS']
df = df[df["SALE_PRICE"].between(100000, 2000000)]
df = df.drop(["EASE-MENT", "ADDRESS", "APARTMENT_NUMBER"], axis = 1)

In [7]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
BUILDING_CLASS_AT_PRESENT          object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

Looks like we need to cast LAND_SQUARE_FEET as an integer.

In [0]:
df["LAND_SQUARE_FEET"] = df["LAND_SQUARE_FEET"].str.replace(",","").astype(int)

In [0]:
categoricals = ["BOROUGH", "NEIGHBORHOOD", "TAX_CLASS_AT_PRESENT", "BUILDING_CLASS_AT_PRESENT", "ZIP_CODE", "BUILDING_CLASS_AT_TIME_OF_SALE"]

In [0]:
df["SALE_DATE"] = pd.to_datetime(df["SALE_DATE"], infer_datetime_format=True)

In [0]:
cutoff = pd.to_datetime("2019-04-01")

In [0]:
train = df[df["SALE_DATE"] < cutoff]
test = df[df["SALE_DATE"] >= cutoff]

In [13]:
df.columns[df.nunique() <= 1]

Index(['BUILDING_CLASS_CATEGORY', 'TAX_CLASS_AT_TIME_OF_SALE'], dtype='object')

Drop all columns which have only one unique value, as they don't provide any extra information.

In [0]:
features = df.columns.drop(["SALE_PRICE", "SALE_DATE", "BUILDING_CLASS_CATEGORY", "TAX_CLASS_AT_TIME_OF_SALE"])
target = "SALE_PRICE"

In [0]:
X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

In [0]:
import category_encoders as ce

In [0]:
encoder = ce.OneHotEncoder(use_cat_names = True, cols = categoricals)

In [0]:
X_train = encoder.fit_transform(X_train)

In [19]:
X_train.shape

(2517, 169)

In [0]:
X_test = encoder.transform(X_test)

In [0]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

In [23]:
for k in range(1, len(X_train.columns) + 1):
  print(f'{k} features')

  selector = SelectKBest(score_func = f_regression, k = k)
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

  model = LinearRegression()
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)
  mae = mean_absolute_error(y_test, y_pred)

  print(f'Test MAE: {mae:.0f} \n')

1 features
Test MAE: 185788 

2 features
Test MAE: 181871 

3 features
Test MAE: 181603 

4 features
Test MAE: 181603 

5 features
Test MAE: 181376 

6 features
Test MAE: 175854 

7 features
Test MAE: 174391 

8 features
Test MAE: 174021 

9 features
Test MAE: 172778 

10 features
Test MAE: 172740 

11 features
Test MAE: 169961 

12 features
Test MAE: 157213 

13 features
Test MAE: 156790 

14 features
Test MAE: 156503 

15 features
Test MAE: 155963 

16 features
Test MAE: 154389 

17 features
Test MAE: 152896 

18 features
Test MAE: 154880 

19 features
Test MAE: 154855 

20 features
Test MAE: 154856 

21 features
Test MAE: 156133 

22 features
Test MAE: 155997 

23 features
Test MAE: 155257 

24 features
Test MAE: 153047 

25 features
Test MAE: 152809 

26 features
Test MAE: 152609 

27 features
Test MAE: 152609 

28 features
Test MAE: 152619 

29 features
Test MAE: 151450 

30 features
Test MAE: 150573 

31 features
Test MAE: 150573 

32 features
Test MAE: 151899 

33 features
Test 

Looks like the optimal model has 98 features. 

In [0]:
selector = SelectKBest(score_func = f_regression, k = 98)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

## Ridge Regression

In [0]:
from sklearn.linear_model import Ridge, RidgeCV

In [0]:
alphas = [.01, .1, .5, 1, 5, 10]

In [39]:
ridge_model = RidgeCV(alphas = alphas, normalize = True, store_cv_values=True)
ridge_model.fit(X_train_selected, y_train)

RidgeCV(alphas=array([ 0.01,  0.1 ,  0.5 ,  1.  ,  5.  , 10.  ]), cv=None,
        fit_intercept=True, gcv_mode=None, normalize=True, scoring=None,
        store_cv_values=True)

In [0]:
y_pred = ridge_model.predict(X_test_selected)
mae = mean_absolute_error(y_test, y_pred)

In [41]:
mae

138286.1364951723

In [42]:
ridge_model.cv_values_

array([[2.11942512e+11, 1.83323936e+11, 1.19863995e+11, 8.43757415e+10,
        2.54126421e+10, 1.49475799e+10],
       [2.92896175e+11, 2.79800618e+11, 2.51860636e+11, 2.35559585e+11,
        2.01298073e+11, 1.91657218e+11],
       [4.27027821e+10, 3.92997717e+10, 3.59223485e+10, 3.52820897e+10,
        3.49698326e+10, 3.51090927e+10],
       ...,
       [1.67786441e+10, 1.46235392e+10, 8.15994281e+09, 4.36698599e+09,
        1.87878709e+08, 4.31356199e+06],
       [6.94651954e+08, 2.12525229e+08, 2.25845915e+08, 1.17192196e+09,
        5.66887454e+09, 7.43698946e+09],
       [1.44325451e+11, 1.50507549e+11, 1.73370276e+11, 1.92128463e+11,
        2.38219574e+11, 2.50853472e+11]])

In [43]:
ridge_model.alpha_

0.1